In [2]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score


In [1]:
# Sigmoid activation
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Sigmoid derivative
def sigmoid_derivative(x):
    return x * (1 - x)


In [3]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):

        # Initialize weights & biases
        self.weights_input_hidden = np.random.uniform(size=(input_size, hidden_size))
        self.weights_hidden_output = np.random.uniform(size=(hidden_size, output_size))

        self.bias_hidden = np.random.uniform(size=(1, hidden_size))
        self.bias_output = np.random.uniform(size=(1, output_size))

    def feedforward(self, X):
        # Input → Hidden
        self.hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = sigmoid(self.hidden_input)

        # Hidden → Output
        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.final_output = sigmoid(self.final_input)

        return self.final_output

    def backpropagation(self, X, y, learning_rate):

        # Error at output
        output_error = y - self.final_output
        output_delta = output_error * sigmoid_derivative(self.final_output)

        # Error at hidden layer
        hidden_error = output_delta.dot(self.weights_hidden_output.T)
        hidden_delta = hidden_error * sigmoid_derivative(self.hidden_output)

        # Update weights & biases
        self.weights_hidden_output += self.hidden_output.T.dot(output_delta) * learning_rate
        self.weights_input_hidden += X.T.dot(hidden_delta) * learning_rate

        self.bias_output += np.sum(output_delta, axis=0, keepdims=True) * learning_rate
        self.bias_hidden += np.sum(hidden_delta, axis=0, keepdims=True) * learning_rate

    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            self.feedforward(X)
            self.backpropagation(X, y, learning_rate)

    def predict(self, X):
        output = self.feedforward(X)
        return np.argmax(output, axis=1)


In [4]:
iris = datasets.load_iris()

X = iris.data
y = iris.target


In [5]:
encoder = OneHotEncoder()
y = encoder.fit_transform(y.reshape(-1, 1)).toarray()


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [7]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [8]:
input_size = X_train.shape[1]     # 4 features
hidden_size = 5                   # hidden neurons
output_size = y_train.shape[1]    # 3 classes

nn = NeuralNetwork(input_size, hidden_size, output_size)

epochs = 10000
learning_rate = 0.01

nn.train(X_train, y_train, epochs, learning_rate)


In [9]:
# Predictions
y_pred = nn.predict(X_test)

# Convert one-hot encoded y_test to labels
y_test_labels = np.argmax(y_test, axis=1)

# Accuracy
accuracy = accuracy_score(y_test_labels, y_pred)

print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 100.00%
